In [1]:
import os
import csv
import cv2
import json
import math
from time import time
import matplotlib.pyplot as plt

In [2]:
from vio_ort import VIO

ModuleNotFoundError: No module named 'modules'

In [ ]:
# Инициализация параметров
odometry = VIO(lat0=54.889668, lon0=83.1258973333, alt0=0)
odometry_org = VIO(lat0=54.889668, lon0=83.1258973333, alt0=0)
set_dir = '2024_12_15_15_31_8_num_3'
json_files = sorted([f for f in os.listdir(set_dir) if f.endswith('.json')])
start = 1000
count_json = 100

In [ ]:
def run_vio(odometry, json_files, start, count_json):
    lat_VIO, lon_VIO, alt_VIO = [], [], []
    for filename in json_files[start:start + count_json]:
        with open(f'{set_dir}/{filename}', 'r') as file:
            data = json.load(file)
            img_path = os.path.join(set_dir, os.path.splitext(filename)[0] + '.jpg')
            image = cv2.imread(img_path)
            result_vio = odometry.add_trace_pt(image, data)
            lat_VIO.append(result_vio['lat'])
            lon_VIO.append(result_vio['lon'])
            alt_VIO.append(result_vio['alt'] * 1000)
    return {
        'lat_VIO': lat_VIO,
        'lon_VIO': lon_VIO,
        'alt_VIO': alt_VIO
    }

In [ ]:
def run_vio_gps(odometry, json_files, start, count_json):
    lat_VIO, lon_VIO, alt_VIO = [], [], []
    lat_GPS, lon_GPS, alt_GPS = [], [], []
    for filename in json_files[start:start + count_json]:
        with open(f'{set_dir}/{filename}', 'r') as file:
            data = json.load(file)
            if 'GNRMC' in data and data['GNRMC']['status'] == 'A':
                img_path = os.path.join(set_dir, os.path.splitext(filename)[0] + '.jpg')
                image = cv2.imread(img_path)
                result_vio = odometry.add_trace_pt(image, data)
                lat_VIO.append(result_vio['lat'])
                lon_VIO.append(result_vio['lon'])
                alt_VIO.append(result_vio['alt'] * 1000)
                lat_GPS.append(data['GNRMC'].get('lat', 0.0))
                lon_GPS.append(data['GNRMC'].get('lon', 0.0))
                alt_GPS.append(data['GPS_RAW_INT']['alt'])
    return {
        'lat_VIO': lat_VIO,
        'lon_VIO': lon_VIO,
        'alt_VIO': alt_VIO,
        'lat_GPS': lat_GPS,
        'lon_GPS': lon_GPS,
        'alt_GPS': alt_GPS,
    }

In [ ]:
def transform_vio_coords(vio_lon_list, vio_lat_list):
    """
    Преобразование координат VIO в новые координаты.
    """
    vio_lon0 = vio_lon_list[0]
    vio_lat0 = vio_lat_list[0]

    vio_lon_range = max(vio_lon_list) - min(vio_lon_list)
    vio_lat_range = max(vio_lat_list) - min(vio_lat_list)

    scale_for_lon = 1 / vio_lat_range  # Широта -> долгота
    scale_for_lat = 1 / vio_lon_range  # Долгота -> широта

    transformed_lon = [(v_lat - vio_lat0) * scale_for_lon + vio_lon0 for v_lat in vio_lat_list]
    transformed_lat = [-(v_lon - vio_lon0) * scale_for_lat + vio_lat0 for v_lon in vio_lon_list]

    return transformed_lon, transformed_lat

In [ ]:
start_time = time()
result = run_vio(odometry, json_files, start, count_json)
elapsed_time = time() - start_time
print(f"Cache time: {elapsed_time:.2f} seconds")

In [ ]:
# Применяем трансформацию координат
transformed_lon, transformed_lat = transform_vio_coords(result['lon_VIO'], result['lat_VIO'])

# Обновляем результаты для построения графиков
result['lon_VIO_transformed'] = transformed_lon
result['lat_VIO_transformed'] = transformed_lat

In [ ]:
# Функция для построения графика с GPS и VIO
def plot_comparison(results):
	vio_lat = results['lat_VIO']
	vio_lon = results['lon_VIO']

	vio_lat_transf = results['lat_VIO_transformed']
	vio_lon_transf = results['lon_VIO_transformed']

	# Построение графиков широты
	plt.figure(figsize=(18, 6))
	plt.plot(vio_lat, vio_lon, linestyle='--')
	plt.plot(vio_lat_transf, vio_lon_transf, linestyle='-.')
	plt.legend()

	plt.tight_layout()
	plt.show()

In [ ]:
plot_comparison(result)